<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [37]:
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup as bs

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import squarify
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_lg')

import gensim
import os, sys, re
# from gensim.models import LdaMulticore
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
yelp = pd.read_json('./data/review_sample.json', lines=True)
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [7]:
def tokenize(doc):
    tokenized_reviews = [token for token in simple_preprocess(doc) if token not in STOPWORDS]
    return tokenized_reviews

In [10]:
yelp['tokenized'] = yelp['text'].apply(lambda x: tokenize(x))
yelp['tokenized'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [came, lunch, togo, service, quick, staff, fri...
2    [ve, vegas, dozens, times, stepped, foot, circ...
3    [went, night, closed, street, party, best, act...
4    [stars, bad, price, lunch, seniors, pay, eatin...
Name: tokenized, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [14]:
vect = TfidfVectorizer()

dtm = vect.fit_transform(yelp['text'].values)
dtm_df = pd.DataFrame(dtm.todense(),columns=vect.get_feature_names())
dtm_df.head()

,00,000,001695,007,00a,00am,00ish,00pm,01,011802,...,誰も乗車しなくても,豆腐花,質問にも丁寧に答えてくれましたし,車好きさんには,這是一個不錯的選擇,運転しない,運転中も英語で指導があります,食べ物はうまい,餐後點了甜點,３時間後の便
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm_df)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [19]:
fake_review_text = ["This restaurant sucks. Food tastes nasty. Environment unsanitary. Bad vibe"]
fake_text_transformed = vect.transform(fake_review_text)
_, similar_topic_indices = nn.kneighbors(fake_text_transformed.todense())

In [22]:
similar_topics = [yelp.text[t] for t in similar_topic_indices]
print(*similar_topics, sep="\n\n")

5423          New owners took over.  The pizza sucks now!
4599    only 1 word describes this place............. ...
3230    This place just sucks. Food was terrible. Serv...
3411    NO RECOMMENDATION TO THIS PLACE AT ALL!!!! SO ...
1356    Cool environment for sports fans. Good food, t...
Name: text, dtype: object


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [27]:
vect_pipe = TfidfVectorizer()
sgdc = SGDClassifier()
pipe = Pipeline([('vect', vect_pipe),
                  ('sgdc', sgdc)])

parameters = {
    'vect__max_df': (0.5, 0.75),
    'vect__min_df': (.02, .05),
    'vect__max_features': ( 100, 500, 1000),
    'sgdc__max_iter': (300, 1000),
}
clf = GridSearchCV(pipe, parameters, cv=5, n_jobs=7)

In [29]:
clf.fit(yelp['text'], yelp['stars'])

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [36]:
print(f'Best score out of grid search is {clf.best_score_}')
clf.predict(yelp['text'])

Best score out of grid search is 0.6121


array([1, 5, 5, ..., 1, 5, 1], dtype=int64)

### Stretch Goal - comparing vectorizor

In [45]:
vect_pipe2 = CountVectorizer()
sgdc2 = SGDClassifier()
pipe2 = Pipeline([('vect', vect_pipe2),
                  ('sgdc', sgdc2)])

parameters2 = {
    'vect__max_df': (0.5, 0.75),
    'vect__min_df': (.02, .05),
    'vect__max_features': ( 100, 500, 1000),
    'sgdc__max_iter': (300, 1000),
}
clf2 = GridSearchCV(pipe2, parameters2, cv=5, n_jobs=7)

In [46]:
clf2.fit(yelp['text'], yelp['stars'])

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [48]:
print(f'Best score out of grid search using TF/IDF vectorizer is {clf.best_score_}')
print(f'Best score out of grid search using Count vectorizer is {clf2.best_score_}')

Best score out of grid search using TF/IDF vectorizer is 0.6121
Best score out of grid search using Count vectorizer is 0.5896


#### From our quick test, we find TF/IDF vectorizer produces higher classification accuracy

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

### Stretch Goal - use generator object

In [30]:
def get_reviews(df, token_col):
    """
    Generator function that takes a dataframe with a tokenized column and yields a tokenized 
    row for processing
    
    Parameters:
    -----------
    df: dataframe
    taken_col: name of the column of tokenized text
    
    Yields:
    -------
    review_token: one record of tokenized review
    """
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token

### 4.1 Estimate a LDA topic model of the review text

Learn the vocubalary of the yelp data:

In [31]:
id2word = corpora.Dictionary(get_reviews(yelp, 'tokenized'))

Create a bag of words representation of the entire corpus

In [39]:
corpus = [id2word.doc2bow(review) for review in get_reviews(yelp, 'tokenized')]

Your LDA model should be ready for estimation: 

In [40]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

### 4.2 Create 1-2 visualizations of the results

In [42]:
topics = [' '.join(re.findall(r'"([^"]*)"',t[1])) for t in lda.print_topics()]
print(*topics, sep = "\n\n")

food good service time great place like ordered got nice

place good great like service time food got ve came

food good great place time like service ve got love

good place great food service time like best ve nice

food place time good great like got service best order

food good service great like place time ve best know

place service great like food good time got staff chicken

great good food place time like service got love nice

food great service place like time good chicken ve experience

food good place like service time great nice best ve


In [43]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-1-nlp-o1ln3r8m\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.001175  0.002788       1        1  10.748766
6      0.005674 -0.003033       2        1  10.635877
1      0.003576  0.001597       3        1  10.314052
9     -0.003637 -0.000729       4        1  10.071934
7     -0.000951  0.002138       5        1   9.986130
3      0.000980  0.004540       6        1   9.920780
0     -0.001607 -0.001416       7        1   9.887290
4     -0.001622 -0.003815       8        1   9.561599
8     -0.000596 -0.001792       9        1   9.478424
5     -0.000641 -0.000279      10        1   9.395144, topic_info=     Category         Freq        Term        Total  loglift  logprob
208   Default  4128.000000       great  4128.000000  30.0000  30.0000
144   Default  4752.000000        food  4752.000000  29.0000  29.0000
133   Default  4557.000000       place  4557.000000  28.0000  28.0000
40    Default  4457.000000        good  4457.000000  27.0000  27.0000
45    Default  3503.000000     service  3503.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
387   Topic10   117.721527       order  1450.104126  -0.1461  -5.9493
381   Topic10   113.799446      little  1373.698486  -0.1258  -5.9832
1277  Topic10   111.635910     chicken  1344.711182  -0.1237  -6.0024
310   Topic10   111.941284         try  1400.445435  -0.1616  -5.9996
358   Topic10   109.329163  definitely  1395.117432  -0.1814  -6.0233

[870 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
1679       1  0.103612        able
1679       2  0.121745        able
1679       3  0.075119        able
1679       4  0.150238        able
1679       5  0.103612        able
...      ...       ...         ...
13097     10  0.032634         yuk
13549      3  0.226941     zombies
13549      9  0.226941     zombies
21303      1  0.621213  とても気に入りました
21304      1  0.620333    探したお店でした

[2831 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 2, 10, 8, 4, 1, 5, 9, 6])

### 4.3 In markdown, write 1-2 paragraphs of analysis on the results of your topic model

Looking at the visualization of the reviews, we find that universally, people are mostly talking about `food`, `great`, `place`, `good`, `service`, `order`, etc. Our guess would be that most reviewers are providing more positive feedback/descriptors in their reviews.  

On the other hand, we are not quite certain if we should remove these overwhelmingly abundant positive descriptors such as `great`, `good`, etc, and extract more information out of the reviews. And if we do that, we would perhaps need to remove the more negative sentiment words that are more of an anomaly. Further testing and iterations are needed.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)